In [1]:
import os

import torch
import transformers
from peft import PeftModel
from transformers import LlamaForCausalLM, LlamaTokenizer  # noqa: F402

BASE_MODEL = "/ibex/user/radhaks/LLMs/LLaMA_7B/Alpaca"

tokenizer = LlamaTokenizer.from_pretrained(BASE_MODEL)

base_model = LlamaForCausalLM.from_pretrained(
    BASE_MODEL,
    load_in_8bit=False,
    torch_dtype=torch.float16,
    device_map={"": "cpu"},)


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /home/radhaks/miniconda3/envs/ALFT/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda117_nocublaslt.so
CUDA SETUP: CUDA runtime path found: /home/radhaks/miniconda3/envs/ALFT/lib/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 7.0
CUDA SETUP: Detected CUDA version 117
CUDA SETUP: Loading binary /home/radhaks/miniconda3/envs/ALFT/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda117_nocublaslt.so...


/home/radhaks/miniconda3/envs/ALFT/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: Compute capability < 7.5 detected! Only slow 8-bit matmul is supported for your GPU!
  warn(msg)


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
# first_weight = base_model.model.layers[0].self_attn.q_proj.weight
# first_weight_old = first_weight.clone()

lora_model = PeftModel.from_pretrained(
    base_model,
    "/ibex/user/radhaks/LLMs/LLaMA_7B/alpaca-lora-srijith/weights/1e-3",
    device_map={"": "cpu"},
    torch_dtype=torch.float16,
)
# lora_weight = lora_model.base_model.model.model.layers[
#     0
# ].self_attn.q_proj.weight
# assert torch.allclose(first_weight_old, first_weight)

In [4]:
# merge weights - new merging method from peft
lora_model = lora_model.merge_and_unload()

In [5]:
!pwd

/ibex/user/radhaks/LLMs/LLaMA_7B/alpaca-lora-srijith


In [8]:
lora_model.train(False)

# did we do anything?
# assert not torch.allclose(first_weight_old, first_weight)
# they are just renaming the weights
lora_model_sd = lora_model.state_dict() 
deloreanized_sd = {
    k.replace("base_model.model.", ""): v
    for k, v in lora_model_sd.items()
    if "lora" not in k
}

LlamaForCausalLM.save_pretrained(
    base_model, "./hf_ckpt", state_dict=deloreanized_sd)


In [5]:
alpaca_model = LlamaForCausalLM.from_pretrained(
    '/ibex/user/radhaks/LLMs/LLaMA_7B/Alpaca',
#     load_in_8bit=False,
       device_map={"": "cpu"},
    torch_dtype=torch.float16,)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [4]:
alpaca_model

NameError: name 'alpaca_model' is not defined

In [6]:
lora_model.state_dict()

OrderedDict([('model.embed_tokens.weight',
              tensor([[ 9.8884e-05, -2.3329e-04,  5.8460e-04,  ..., -3.4237e-04,
                        5.9724e-05, -1.1957e-04],
                      [ 1.4793e-02, -1.2634e-02,  1.2726e-02,  ...,  1.3229e-02,
                        7.2594e-03, -4.4703e-04],
                      [ 1.7433e-03,  1.7633e-03, -1.4465e-02,  ..., -1.1444e-02,
                       -1.2665e-02,  3.7289e-04],
                      ...,
                      [-1.1368e-02, -1.4801e-02, -3.5667e-03,  ...,  6.5308e-03,
                       -2.2263e-02, -6.1455e-03],
                      [-1.3992e-02,  1.6985e-03, -2.1469e-02,  ...,  1.3527e-02,
                        2.8290e-02, -8.9111e-03],
                      [-1.5526e-03, -2.6894e-04,  8.6641e-04,  ...,  2.1591e-03,
                       -6.6280e-04, -3.1090e-03]], dtype=torch.float16)),
             ('model.layers.0.self_attn.q_proj.weight',
              tensor([[-0.0069, -0.0299,  0.0064,  ...,  0.0165,

In [7]:
alpaca_model.state_dict()

OrderedDict([('model.embed_tokens.weight',
              tensor([[ 9.8884e-05, -2.3329e-04,  5.8460e-04,  ..., -3.4237e-04,
                        5.9724e-05, -1.1957e-04],
                      [ 1.4793e-02, -1.2634e-02,  1.2726e-02,  ...,  1.3229e-02,
                        7.2594e-03, -4.4703e-04],
                      [ 1.7433e-03,  1.7633e-03, -1.4465e-02,  ..., -1.1444e-02,
                       -1.2665e-02,  3.7289e-04],
                      ...,
                      [-1.1368e-02, -1.4801e-02, -3.5667e-03,  ...,  6.5308e-03,
                       -2.2263e-02, -6.1455e-03],
                      [-1.3992e-02,  1.6985e-03, -2.1469e-02,  ...,  1.3527e-02,
                        2.8290e-02, -8.9111e-03],
                      [-1.5526e-03, -2.6894e-04,  8.6641e-04,  ...,  2.1591e-03,
                       -6.6280e-04, -3.1090e-03]], dtype=torch.float16)),
             ('model.layers.0.self_attn.q_proj.weight',
              tensor([[-0.0090, -0.0300,  0.0084,  ...,  0.0182,